In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
dat = pd.read_csv('train.csv', index_col='row_id')

the test data is on a Monday afternoon, produce two test sets in Monday after for checking

In [3]:
test_periods = [
    ['1991-08-26 12:00:00', '1991-09-02 24:00:00'],
    ['1991-09-02 12:00:00', '1991-09-02 24:00:00'],
    ['1991-09-09 12:00:00', '1991-09-09 24:00:00'],
    ['1991-09-16 12:00:00', '1991-09-16 24:00:00']]

In [4]:
def preprocess(dat):
    time_mapper = {}
    ii = 0
    for h in range(24):
        for mm in ['00','20','40']:
            hh = '{0:02d}'.format(h)
            time_mapper[hh+':'+mm] = ii
            ii += 1

    dat['unique'] = dat['x'].astype(str) + dat['y'].astype(str) + dat['direction']
    dat['day'] = pd.to_datetime(dat['time']).dt.weekday
    dat['time_stamp'] = dat['time'].apply(lambda x:time_mapper[x.split()[1][:5]])

    tmp = dat.groupby(['unique','day','time_stamp']).agg({'congestion':np.median})
    median_mapper = tmp.to_dict()['congestion']
    
    congestions = dat[['unique','congestion','time']].set_index('time')
    congestions = congestions.groupby(['unique','time']).mean().unstack('unique')
    congestions.columns = congestions.columns.droplevel(0)
    
    cols = congestions.columns.to_list()
    for col in cols:
        col_new = col + '-1'
        congestions[col_new] = np.hstack((0, congestions[col][:-1]))
        col_new = col + '-2'
        congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
    congestions = congestions.copy()
    congestions = congestions.drop(index=congestions.index[:2], axis=1)
    
    congestions['time_stamp'] = \
       pd.Series(congestions.index).apply(lambda x:time_mapper[x.split()[1][:5]]).values
    congestions['day'] = \
       pd.to_datetime(pd.Series(congestions.index)).dt.weekday.values
    
    return congestions, median_mapper, time_mapper

In [5]:
congestions, median_mapper, time_mapper = preprocess(dat)

/tmp/ipykernel_29401/3096527658.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, 0, congestions[col][:-2]))
/tmp/ipykernel_29401/3096527658.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = np.hstack((0, congestions[col][:-1]))


In [6]:
uniques = dat['unique'].unique()
print(uniques,len(uniques))

['00EB' '00NB' '00SB' '01EB' '01NB' '01SB' '01WB' '02EB' '02NB' '02SB'
 '02WB' '03EB' '03NB' '03NE' '03SB' '03SW' '03WB' '10EB' '10NB' '10NE'
 '10SB' '10SW' '10WB' '11EB' '11NB' '11SB' '11WB' '12EB' '12NB' '12NE'
 '12SB' '12SW' '12WB' '13EB' '13NB' '13NE' '13SB' '13SW' '13WB' '20EB'
 '20NB' '20SB' '20WB' '21EB' '21NB' '21NE' '21NW' '21SB' '21SE' '21SW'
 '21WB' '22EB' '22NB' '22NE' '22NW' '22SB' '22SE' '22SW' '22WB' '23EB'
 '23NB' '23NE' '23SB' '23SW' '23WB'] 65


In [7]:
congestions

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-04-01 00:40:00,70.0,49.0,24.0,31.0,51.0,57.0,91.0,42.0,41.0,35.0,...,30.0,30.0,80.0,70.0,29.0,29.0,57.0,26.0,2,0
1991-04-01 01:00:00,70.0,49.0,24.0,36.0,51.0,46.0,49.0,39.0,44.0,58.0,...,30.0,30.0,79.0,80.0,29.0,29.0,48.0,57.0,3,0
1991-04-01 01:20:00,70.0,49.0,24.0,29.0,53.0,83.0,62.0,36.0,46.0,46.0,...,30.0,30.0,77.0,79.0,29.0,29.0,44.0,48.0,4,0
1991-04-01 01:40:00,70.0,49.0,24.0,15.0,48.0,38.0,39.0,36.0,51.0,43.0,...,30.0,30.0,79.0,77.0,29.0,29.0,44.0,44.0,5,0
1991-04-01 02:00:00,70.0,49.0,65.0,20.0,27.0,46.0,39.0,35.0,56.0,49.0,...,30.0,30.0,85.0,79.0,29.0,29.0,44.0,44.0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-09-30 10:20:00,43.0,29.0,60.0,23.0,63.0,61.0,42.0,47.0,51.0,56.0,...,25.0,31.0,69.0,66.0,24.0,35.0,33.0,41.0,31,0
1991-09-30 10:40:00,47.0,28.0,35.0,24.0,62.0,63.0,32.0,44.0,57.0,65.0,...,29.0,25.0,68.0,69.0,22.0,24.0,49.0,33.0,32,0
1991-09-30 11:00:00,40.0,31.0,53.0,22.0,65.0,58.0,38.0,44.0,60.0,62.0,...,25.0,29.0,70.0,68.0,20.0,22.0,47.0,49.0,33,0


In [8]:
def assemble(col):
    cols = [col for col in congestions.columns.to_list() if '-' in col]
    cols.append('time_stamp')
    cols.append('day')
    dat = congestions[cols].copy()  #the copy is needed
    dat['median'] = dat.apply(\
                        lambda x: median_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
    dat['target'] = congestions[col].values
    
    trains, tests = [], []
    for period in test_periods:
        trains.append(dat[dat.index < period[0]])
        tests.append(dat[(dat.index >= period[0]) & (dat.index <= period[1])])
    
    return trains, tests, dat

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae

In [15]:
regrs = {}

for col in uniques:
    trains, tests, train_all = assemble(col)
    
    best_mae = 1000.0
    print(f"fitting {col}")
    for leaf in [10,20,50,100,200]:
        regr = RandomForestRegressor(n_estimators=100, min_samples_leaf=leaf, criterion='absolute_error', n_jobs=48)
        
        results = []
        for train, test in zip(trains, tests):
            X, y = train.drop(columns='target').values, train['target'].values
            regr.fit(X, y)
            
            X, y_test = test.drop(columns='target').values, test['target'].values
            y_pred = regr.predict(X)
            
            results.append(mae(y_pred, y_test))
        
        mean_mae = np.mean(results)
        print(f"{col} leaf {leaf}: {mean_mae}")
        if mean_mae < best_mae:
            best_leaf = leaf
            best_mae = mean_mae
    
    print(f"refitting with best leaf={best_leaf}")
    regr = RandomForestRegressor(n_estimators=100, min_samples_leaf=best_leaf, criterion='absolute_error', n_jobs=48)
    X, y = train_all.drop(columns='target').values, train_all['target'].values
    regr.fit(X,y)
    regrs[col] = regr

fitting 00EB
00EB leaf 10: 9.07222631495564
00EB leaf 20: 9.071987484157159
00EB leaf 50: 9.095085287283482
00EB leaf 100: 9.070248996620196
00EB leaf 200: 9.06383819180397
refitting with best leaf=200
fitting 00NB
00NB leaf 10: 7.4497983998732575
00NB leaf 20: 7.485050168990283
00NB leaf 50: 7.462046234685256
00NB leaf 100: 7.492965515420364
00NB leaf 200: 7.494356516687791
refitting with best leaf=10
fitting 00SB
00SB leaf 10: 8.194857810519645
00SB leaf 20: 8.185807852767216
00SB leaf 50: 8.220774186734262
00SB leaf 100: 8.320138888888888
00SB leaf 200: 8.36553192332066
refitting with best leaf=20
fitting 01EB
01EB leaf 10: 4.706972565483735
01EB leaf 20: 4.671289607097592
01EB leaf 50: 4.763117474651458
01EB leaf 100: 4.770992289818336
01EB leaf 200: 4.845978955428813
refitting with best leaf=20
fitting 01NB
01NB leaf 10: 4.754151087874947
01NB leaf 20: 4.918940642163076
01NB leaf 50: 5.118374128643853
01NB leaf 100: 5.186009980988593
01NB leaf 200: 5.303563846641318
refitting with

20NB leaf 50: 4.563609790874525
20NB leaf 100: 4.6188433407266585
20NB leaf 200: 4.634592442965779
refitting with best leaf=10
fitting 20SB
20SB leaf 10: 5.293594476130123
20SB leaf 20: 5.262001742712294
20SB leaf 50: 5.278318942754542
20SB leaf 100: 5.284105539712717
20SB leaf 200: 5.323214115969582
refitting with best leaf=20
fitting 20WB
20WB leaf 10: 4.692648262568652
20WB leaf 20: 4.7063838719898605
20WB leaf 50: 4.742547000422476
20WB leaf 100: 4.792354641951838
20WB leaf 200: 4.885486111111111
refitting with best leaf=10
fitting 21EB
21EB leaf 10: 5.889970162653147
21EB leaf 20: 5.9522818969159275
21EB leaf 50: 5.949392955217576
21EB leaf 100: 5.996771889522603
21EB leaf 200: 6.040468948035487
refitting with best leaf=10
fitting 21NB
21NB leaf 10: 4.2648777460921
21NB leaf 20: 4.329670600971694
21NB leaf 50: 4.3256009716941275
21NB leaf 100: 4.425749102239121
21NB leaf 200: 4.525365705534432
refitting with best leaf=10
fitting 21NE
21NE leaf 10: 3.656529362061681
21NE leaf 20: 3

In [16]:
import pickle

In [17]:
with open('baseline1_models.pickle', 'wb') as fid:
    pickle.dump(regrs, fid)

In [18]:
dat.head()

,time,x,y,direction,congestion,unique,day,time_stamp
row_id,,,,,,,,
0,1991-04-01 00:00:00,0,0,EB,70,00EB,0,0
1,1991-04-01 00:00:00,0,0,NB,49,00NB,0,0
2,1991-04-01 00:00:00,0,0,SB,24,00SB,0,0
3,1991-04-01 00:00:00,0,1,EB,18,01EB,0,0
4,1991-04-01 00:00:00,0,1,NB,60,01NB,0,0


In [22]:
test = pd.read_csv('test.csv')
test.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


In [23]:
fake_test = dat[(dat['time']>=test_periods[-1][0]) & (dat['time']<=test_periods[-1][1])]
fake_test

,time,x,y,direction,congestion,unique,day,time_stamp
row_id,,,,,,,,
784875,1991-09-16 12:00:00,0,0,EB,40,00EB,0,36
784876,1991-09-16 12:00:00,0,0,NB,38,00NB,0,36
784877,1991-09-16 12:00:00,0,0,SB,60,00SB,0,36
784878,1991-09-16 12:00:00,0,1,EB,27,01EB,0,36
784879,1991-09-16 12:00:00,0,1,NB,76,01NB,0,36
...,...,...,...,...,...,...,...,...
787210,1991-09-16 23:40:00,2,3,NB,65,23NB,0,71
787211,1991-09-16 23:40:00,2,3,NE,18,23NE,0,71
787212,1991-09-16 23:40:00,2,3,SB,73,23SB,0,71


In [24]:
def preprocess_test(dat, time_mapper):
    dat['unique'] = dat['x'].astype(str) + dat['y'].astype(str) + dat['direction']
    dat['day'] = pd.to_datetime(dat['time']).dt.weekday
    dat['time_stamp'] = dat['time'].apply(lambda x:time_mapper[x.split()[1][:5]])
    congestions = dat[['unique','time']].set_index('time')
    congestions['congestion'] = 0.0
    congestions = congestions.groupby(['unique','time']).mean().unstack('unique')
    congestions.columns = congestions.columns.droplevel(0)
    
    for col in congestions.columns.to_list():
        col_new = col + '-1'
        congestions[col_new] = 0.0
        col_new = col + '-2'
        congestions[col_new] = 0.0
    congestions = congestions.copy()
    
    congestions['time_stamp'] = \
       pd.Series(congestions.index).apply(lambda x:time_mapper[x.split()[1][:5]]).values
    congestions['day'] = \
       pd.to_datetime(pd.Series(congestions.index)).dt.weekday.values
    
    return congestions

In [25]:
test = fake_test.copy()

In [26]:
congestions_test = preprocess_test(test, time_mapper)
congestions_test

/tmp/ipykernel_29401/699427706.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = 0.0
/tmp/ipykernel_29401/699427706.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = 0.0


unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-09-16 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,0
1991-09-16 12:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,0
1991-09-16 12:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,0
1991-09-16 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,0
1991-09-16 13:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,0
1991-09-16 13:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41,0
1991-09-16 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42,0
1991-09-16 14:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,0
1991-09-16 14:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44,0


In [27]:
congestions_test

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-09-16 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,0
1991-09-16 12:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,0
1991-09-16 12:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,0
1991-09-16 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,0
1991-09-16 13:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,0
1991-09-16 13:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41,0
1991-09-16 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42,0
1991-09-16 14:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,0
1991-09-16 14:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44,0


In [28]:
col_mapper = {}
for col in uniques:
    col_mapper[col+'-1'] = col
    col_mapper[col+'-2'] = col+'-1'

In [29]:
idx_test = congestions_test.index[0]
idx_train = congestions.index[-1]
for col in uniques:
    col1, col2 = col+'-1', col+'-2'
    congestions_test.at[idx_test, col1] = congestions.at[idx_train, col_mapper[col1]]
    congestions_test.at[idx_test, col2] = congestions.at[idx_train, col_mapper[col2]]
congestions_test.loc[idx_test]

unique
00EB           0.0
00NB           0.0
00SB           0.0
01EB           0.0
01NB           0.0
              ... 
23SW-2        22.0
23WB-1        24.0
23WB-2        53.0
time_stamp    36.0
day            0.0
Name: 1991-09-16 12:00:00, Length: 197, dtype: float64

In [30]:
def predict():
    cols = [col for col in congestions.columns.to_list() if '-' in col]
    cols.append('time_stamp')
    cols.append('day')
    
    for i in range(len(congestions_test.index)):
        
        # fill the new row
        idx = congestions_test.index[i]
        if (i>0):
            idx1 = congestions_test.index[i-1]
            for col in uniques:
                col1, col2 = col+'-1', col+'-2'
                congestions_test.at[idx, col1] = congestions_test.at[idx1, col_mapper[col1]]
                congestions_test.at[idx, col2] = congestions_test.at[idx1, col_mapper[col2]]
        
        # make prediction of this row
        for col in uniques:
            dat = congestions_test.loc[[idx], cols]
            dat['median'] = dat.apply(\
                        lambda x: median_mapper[(col, x['day'], x['time_stamp'])], axis=1).values
            y = regrs[col].predict(dat.values)
            congestions_test.at[idx, col] = y

In [31]:
predict()

In [32]:
congestions_test

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-09-16 12:00:00,49.230,35.575,53.745,22.710,69.555,60.900,43.935,47.510,60.875,71.295,...,28.000,22.000,68.000,69.000,17.000,22.000,24.000,53.000,36,0
1991-09-16 12:20:00,46.520,35.560,52.220,26.675,70.725,62.905,42.440,49.545,58.930,66.545,...,27.090,28.000,69.905,68.000,18.435,17.000,46.070,24.000,37,0
1991-09-16 12:40:00,47.330,36.470,54.015,27.085,72.100,62.505,40.445,49.030,57.910,64.295,...,27.370,27.090,70.030,69.905,19.990,18.435,41.495,46.070,38,0
1991-09-16 13:00:00,49.370,36.875,52.365,26.765,72.570,62.940,46.100,47.260,61.425,63.230,...,26.030,27.370,70.740,70.030,15.840,19.990,47.550,41.495,39,0
1991-09-16 13:20:00,46.490,36.710,49.755,29.940,73.070,63.065,45.220,48.860,62.495,64.075,...,25.380,26.030,70.450,70.740,17.940,15.840,46.445,47.550,40,0
1991-09-16 13:40:00,52.270,41.735,52.510,29.025,75.550,63.140,47.585,50.210,64.290,64.220,...,26.810,25.380,70.010,70.450,15.820,17.940,38.945,46.445,41,0
1991-09-16 14:00:00,50.800,44.765,53.835,28.540,77.160,63.205,48.420,50.695,64.785,64.440,...,28.040,26.810,70.010,70.010,16.330,15.820,41.785,38.945,42,0
1991-09-16 14:20:00,50.240,40.535,52.640,30.885,75.735,63.510,48.990,51.170,64.775,67.370,...,27.090,28.040,70.010,70.010,15.770,16.330,41.850,41.785,43,0
1991-09-16 14:40:00,52.610,41.565,52.355,32.375,77.725,65.345,48.025,50.740,66.045,69.120,...,26.980,27.090,70.370,70.010,15.810,15.770,46.180,41.850,44,0


In [33]:
congestions_test[['00EB','00EB-1','00EB-2']]

unique,00EB,00EB-1,00EB-2
time,,,
1991-09-16 12:00:00,49.230,70.000,52.000
1991-09-16 12:20:00,46.520,49.230,70.000
1991-09-16 12:40:00,47.330,46.520,49.230
1991-09-16 13:00:00,49.370,47.330,46.520
1991-09-16 13:20:00,46.490,49.370,47.330
1991-09-16 13:40:00,52.270,46.490,49.370
1991-09-16 14:00:00,50.800,52.270,46.490
1991-09-16 14:20:00,50.240,50.800,52.270
1991-09-16 14:40:00,52.610,50.240,50.800


In [34]:
test.head()

,time,x,y,direction,congestion,unique,day,time_stamp
row_id,,,,,,,,
784875,1991-09-16 12:00:00,0,0,EB,40,00EB,0,36
784876,1991-09-16 12:00:00,0,0,NB,38,00NB,0,36
784877,1991-09-16 12:00:00,0,0,SB,60,00SB,0,36
784878,1991-09-16 12:00:00,0,1,EB,27,01EB,0,36
784879,1991-09-16 12:00:00,0,1,NB,76,01NB,0,36


In [37]:
test['congestion_pred'] = 0.0
for idx in test.index:
    time, unique = test.at[idx,'time'], test.at[idx,'unique']
    test.at[idx,'congestion_pred'] = congestions_test.at[time,unique]
test

,time,x,y,direction,congestion,unique,day,time_stamp,congestion_pred
row_id,,,,,,,,,
784875,1991-09-16 12:00:00,0,0,EB,40,00EB,0,36,49.230
784876,1991-09-16 12:00:00,0,0,NB,38,00NB,0,36,35.575
784877,1991-09-16 12:00:00,0,0,SB,60,00SB,0,36,53.745
784878,1991-09-16 12:00:00,0,1,EB,27,01EB,0,36,22.710
784879,1991-09-16 12:00:00,0,1,NB,76,01NB,0,36,69.555
...,...,...,...,...,...,...,...,...,...
787210,1991-09-16 23:40:00,2,3,NB,65,23NB,0,71,67.495
787211,1991-09-16 23:40:00,2,3,NE,18,23NE,0,71,25.815
787212,1991-09-16 23:40:00,2,3,SB,73,23SB,0,71,70.755


In [40]:
mae(test['congestion'], test['congestion_pred'].astype(int))

5.884188034188035

In [42]:
test = pd.read_csv('test.csv')
congestions_test = preprocess_test(test, time_mapper)
congestions_test

/tmp/ipykernel_29401/699427706.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = 0.0
/tmp/ipykernel_29401/699427706.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  congestions[col_new] = 0.0


unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,0
1991-09-30 12:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,0
1991-09-30 12:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,0
1991-09-30 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,0
1991-09-30 13:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,0
1991-09-30 13:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41,0
1991-09-30 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42,0
1991-09-30 14:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,0
1991-09-30 14:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44,0


In [43]:
congestions.tail()

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 10:20:00,43.0,29.0,60.0,23.0,63.0,61.0,42.0,47.0,51.0,56.0,...,25.0,31.0,69.0,66.0,24.0,35.0,33.0,41.0,31,0
1991-09-30 10:40:00,47.0,28.0,35.0,24.0,62.0,63.0,32.0,44.0,57.0,65.0,...,29.0,25.0,68.0,69.0,22.0,24.0,49.0,33.0,32,0
1991-09-30 11:00:00,40.0,31.0,53.0,22.0,65.0,58.0,38.0,44.0,60.0,62.0,...,25.0,29.0,70.0,68.0,20.0,22.0,47.0,49.0,33,0
1991-09-30 11:20:00,52.0,36.0,59.0,35.0,67.0,54.0,26.0,44.0,59.0,57.0,...,29.0,25.0,69.0,70.0,19.0,20.0,37.0,47.0,34,0
1991-09-30 11:40:00,70.0,40.0,51.0,24.0,65.0,52.0,42.0,43.0,62.0,77.0,...,22.0,29.0,69.0,69.0,22.0,19.0,53.0,37.0,35,0


In [45]:
idx_test = congestions_test.index[0]
idx_train = congestions.index[-1]
for col in uniques:
    col1, col2 = col+'-1', col+'-2'
    congestions_test.at[idx_test, col1] = congestions.at[idx_train, col_mapper[col1]]
    congestions_test.at[idx_test, col2] = congestions.at[idx_train, col_mapper[col2]]
congestions_test.head()

unique,00EB,00NB,00SB,01EB,01NB,01SB,01WB,02EB,02NB,02SB,...,23NE-1,23NE-2,23SB-1,23SB-2,23SW-1,23SW-2,23WB-1,23WB-2,time_stamp,day
time,,,,,,,,,,,,,,,,,,,,,
1991-09-30 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,22.0,68.0,69.0,17.0,22.0,24.0,53.0,36,0
1991-09-30 12:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,0
1991-09-30 12:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38,0
1991-09-30 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,0
1991-09-30 13:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,0


In [46]:
predict()

In [47]:
test.head()

,row_id,time,x,y,direction,unique,day,time_stamp
0,848835,1991-09-30 12:00:00,0,0,EB,00EB,0,36
1,848836,1991-09-30 12:00:00,0,0,NB,00NB,0,36
2,848837,1991-09-30 12:00:00,0,0,SB,00SB,0,36
3,848838,1991-09-30 12:00:00,0,1,EB,01EB,0,36
4,848839,1991-09-30 12:00:00,0,1,NB,01NB,0,36


In [48]:
test['congestion'] = 0.0
for idx in test.index:
    time, unique = test.at[idx,'time'], test.at[idx,'unique']
    test.at[idx,'congestion'] = congestions_test.at[time,unique]
test.head()

,row_id,time,x,y,direction,unique,day,time_stamp,congestion
0,848835,1991-09-30 12:00:00,0,0,EB,00EB,0,36,49.230
1,848836,1991-09-30 12:00:00,0,0,NB,00NB,0,36,35.575
2,848837,1991-09-30 12:00:00,0,0,SB,00SB,0,36,53.745
3,848838,1991-09-30 12:00:00,0,1,EB,01EB,0,36,22.710
4,848839,1991-09-30 12:00:00,0,1,NB,01NB,0,36,69.555


In [49]:
test[['row_id', 'congestion']].to_csv('submission_baseline1.csv', index=False)

In [50]:
dat.head()

,time,x,y,direction,congestion,unique,day,time_stamp
row_id,,,,,,,,
0,1991-04-01 00:00:00,0,0,EB,70,00EB,0,0
1,1991-04-01 00:00:00,0,0,NB,49,00NB,0,0
2,1991-04-01 00:00:00,0,0,SB,24,00SB,0,0
3,1991-04-01 00:00:00,0,1,EB,18,01EB,0,0
4,1991-04-01 00:00:00,0,1,NB,60,01NB,0,0


In [51]:
lower = dat.groupby(['unique','day','time_stamp'])['congestion'].quantile(0.15).to_dict()
upper = dat.groupby(['unique','day','time_stamp'])['congestion'].quantile(0.7).to_dict()

In [52]:
lower

{('00EB', 0, 0): 33.0,
 ('00EB', 0, 1): 33.0,
 ('00EB', 0, 2): 33.0,
 ('00EB', 0, 3): 33.0,
 ('00EB', 0, 4): 33.0,
 ('00EB', 0, 5): 33.0,
 ('00EB', 0, 6): 42.4,
 ('00EB', 0, 7): 42.4,
 ('00EB', 0, 8): 42.4,
 ('00EB', 0, 9): 42.4,
 ('00EB', 0, 10): 42.4,
 ('00EB', 0, 11): 42.4,
 ('00EB', 0, 12): 42.4,
 ('00EB', 0, 13): 42.4,
 ('00EB', 0, 14): 34.8,
 ('00EB', 0, 15): 42.9,
 ('00EB', 0, 16): 40.7,
 ('00EB', 0, 17): 37.0,
 ('00EB', 0, 18): 43.0,
 ('00EB', 0, 19): 37.0,
 ('00EB', 0, 20): 39.8,
 ('00EB', 0, 21): 31.8,
 ('00EB', 0, 22): 40.0,
 ('00EB', 0, 23): 37.0,
 ('00EB', 0, 24): 37.0,
 ('00EB', 0, 25): 30.0,
 ('00EB', 0, 26): 40.0,
 ('00EB', 0, 27): 39.7,
 ('00EB', 0, 28): 37.0,
 ('00EB', 0, 29): 42.9,
 ('00EB', 0, 30): 37.9,
 ('00EB', 0, 31): 39.7,
 ('00EB', 0, 32): 46.8,
 ('00EB', 0, 33): 38.0,
 ('00EB', 0, 34): 42.0,
 ('00EB', 0, 35): 46.6,
 ('00EB', 0, 36): 37.0,
 ('00EB', 0, 37): 35.0,
 ('00EB', 0, 38): 37.0,
 ('00EB', 0, 39): 41.5,
 ('00EB', 0, 40): 39.25,
 ('00EB', 0, 41): 39.5,
 

In [53]:
test

,row_id,time,x,y,direction,unique,day,time_stamp,congestion
0,848835,1991-09-30 12:00:00,0,0,EB,00EB,0,36,49.230
1,848836,1991-09-30 12:00:00,0,0,NB,00NB,0,36,35.575
2,848837,1991-09-30 12:00:00,0,0,SB,00SB,0,36,53.745
3,848838,1991-09-30 12:00:00,0,1,EB,01EB,0,36,22.710
4,848839,1991-09-30 12:00:00,0,1,NB,01NB,0,36,69.555
...,...,...,...,...,...,...,...,...,...
2335,851170,1991-09-30 23:40:00,2,3,NB,23NB,0,71,67.495
2336,851171,1991-09-30 23:40:00,2,3,NE,23NE,0,71,25.815
2337,851172,1991-09-30 23:40:00,2,3,SB,23SB,0,71,70.755
2338,851173,1991-09-30 23:40:00,2,3,SW,23SW,0,71,13.850


In [65]:
test['lower'] = test.apply(lambda x:\
    lower[(x['unique'], x['day'], x['time_stamp'])], axis=1)
test['upper'] = test.apply(lambda x:\
    upper[(x['unique'], x['day'], x['time_stamp'])], axis=1)
test

,row_id,time,x,y,direction,unique,day,time_stamp,congestion,lower,upper
0,848835,1991-09-30 12:00:00,0,0,EB,00EB,0,36,49.230,37.00,50.0
1,848836,1991-09-30 12:00:00,0,0,NB,00NB,0,36,35.575,28.75,38.0
2,848837,1991-09-30 12:00:00,0,0,SB,00SB,0,36,53.745,50.00,58.0
3,848838,1991-09-30 12:00:00,0,1,EB,01EB,0,36,22.710,18.75,24.5
4,848839,1991-09-30 12:00:00,0,1,NB,01NB,0,36,69.555,65.75,74.5
...,...,...,...,...,...,...,...,...,...,...,...
2335,851170,1991-09-30 23:40:00,2,3,NB,23NB,0,71,67.495,64.00,70.0
2336,851171,1991-09-30 23:40:00,2,3,NE,23NE,0,71,25.815,25.00,35.0
2337,851172,1991-09-30 23:40:00,2,3,SB,23SB,0,71,70.755,69.00,72.0
2338,851173,1991-09-30 23:40:00,2,3,SW,23SW,0,71,13.850,11.00,18.0


In [68]:
test['congestion_tweak'] = test['congestion'].clip(test['lower'],test['upper'])

In [69]:
test[['row_id', 'congestion_tweak']].to_csv('submission_baseline1_tweaked.csv', index=False)